In [3]:
import xml.etree.ElementTree as ET
from collections import defaultdict

# Load and parse the XML file
tree = ET.parse('./pit-reports-Manual/mutations.xml')  # replace with your actual file path
root = tree.getroot()

# Dictionary to track counts per mutator
mutator_stats = defaultdict(lambda: {'generated': 0, 'killed': 0, 'survived': 0})

# Loop through each mutation element
for mutation in root.findall('mutation'):
    mutator = mutation.find('mutator').text
    status = mutation.get('status')

    # Update total count for this mutator type
    mutator_stats[mutator]['generated'] += 1

    # Update kill/survive count based on status
    if status == 'KILLED':
        mutator_stats[mutator]['killed'] += 1
    else:
        mutator_stats[mutator]['survived'] += 1

# Print the results in table format
print(f"{'Mutant Type':35} {'# Generated':>12} {'# Killed':>10} {'# Survived':>12}")
print("-" * 70)
for mutator, stats in mutator_stats.items():
    print(f"{mutator:35} {stats['generated']:>12} {stats['killed']:>10} {stats['survived']:>12}")


Mutant Type                          # Generated   # Killed   # Survived
----------------------------------------------------------------------
org.pitest.mutationtest.engine.gregor.mutators.ConditionalsBoundaryMutator            4          0            4
org.pitest.mutationtest.engine.gregor.mutators.NegateConditionalsMutator           21         13            8
org.pitest.mutationtest.engine.gregor.mutators.VoidMethodCallMutator            3          2            1
org.pitest.mutationtest.engine.gregor.mutators.returns.NullReturnValsMutator           14          6            8
org.pitest.mutationtest.engine.gregor.mutators.MathMutator            2          2            0
org.pitest.mutationtest.engine.gregor.mutators.returns.PrimitiveReturnsMutator            4          4            0
org.pitest.mutationtest.engine.gregor.mutators.returns.BooleanTrueReturnValsMutator            7          3            4
org.pitest.mutationtest.engine.gregor.mutators.returns.BooleanFalseReturnValsMuta

In [9]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict

def parse_mutation_report(xml_file):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Dictionary to hold mutator stats
    mutator_stats = defaultdict(lambda: {
        'generated': 0,
        'killed': 0,
        'survived': 0,
        'no_coverage': 0,
        'timed_out': 0,
        'memory_error': 0
    })

    # Process each mutation entry
    for mutation in root.findall('mutation'):
        full_mutator = mutation.find('mutator').text
        mutator = full_mutator.split('.')[-1]
        status = mutation.get('status')

        stats = mutator_stats[mutator]
        stats['generated'] += 1

        if status == 'KILLED':
            stats['killed'] += 1
        else:
            stats['survived'] += 1

        if status == 'NO_COVERAGE':
            stats['no_coverage'] += 1
        elif status == 'TIMED_OUT':
            stats['timed_out'] += 1
        elif status in ['MEMORY_ERROR', 'RUN_ERROR']:
            stats['memory_error'] += 1

    # Convert to DataFrame
    df = pd.DataFrame([
        {
            'Mutant Type': mutator,
            '# Generated': stats['generated'],
            '# Killed': stats['killed'],
            '# Survived': stats['survived'],
            'No Coverage': stats['no_coverage'],
            'Timed Out': stats['timed_out'],
            'Memory / Run Error': stats['memory_error']
        }
        for mutator, stats in mutator_stats.items()
    ])

    # Sort by # Generated
    df = df.sort_values(by='# Generated', ascending=False).reset_index(drop=True)
    return df

# Example usage:
# df = parse_mutation_report('mutations.xml')
# print(df)


In [10]:
df = parse_mutation_report('./pit-reports-Manual/mutations.xml')
df

,Mutant Type,# Generated,# Killed,# Survived,No Coverage,Timed Out,Memory / Run Error
0,NegateConditionalsMutator,21,13,8,8,0,0
1,NullReturnValsMutator,14,6,8,8,0,0
2,BooleanTrueReturnValsMutator,7,3,4,1,0,0
3,ConditionalsBoundaryMutator,4,0,4,2,0,0
4,PrimitiveReturnsMutator,4,4,0,0,0,0
5,VoidMethodCallMutator,3,2,1,1,0,0
6,MathMutator,2,2,0,0,0,0
7,BooleanFalseReturnValsMutator,2,2,0,0,0,0


In [11]:
df = parse_mutation_report('./pit-reports-ChatGPT/mutations.xml')
df

,Mutant Type,# Generated,# Killed,# Survived,No Coverage,Timed Out,Memory / Run Error
0,NegateConditionalsMutator,21,21,0,0,0,0
1,NullReturnValsMutator,14,13,1,1,0,0
2,BooleanTrueReturnValsMutator,7,3,4,0,0,0
3,ConditionalsBoundaryMutator,4,0,4,0,0,0
4,PrimitiveReturnsMutator,4,4,0,0,0,0
5,VoidMethodCallMutator,3,3,0,0,0,0
6,MathMutator,2,2,0,0,0,0
7,BooleanFalseReturnValsMutator,2,2,0,0,0,0


In [12]:
df = parse_mutation_report('./pit-reports-Deepseek/mutations.xml')
df

,Mutant Type,# Generated,# Killed,# Survived,No Coverage,Timed Out,Memory / Run Error
0,NegateConditionalsMutator,21,21,0,0,0,0
1,NullReturnValsMutator,14,11,3,3,0,0
2,BooleanTrueReturnValsMutator,7,4,3,0,0,0
3,ConditionalsBoundaryMutator,4,0,4,0,0,0
4,PrimitiveReturnsMutator,4,4,0,0,0,0
5,VoidMethodCallMutator,3,3,0,0,0,0
6,MathMutator,2,2,0,0,0,0
7,BooleanFalseReturnValsMutator,2,2,0,0,0,0
